## 関数定義

- ### スクレイピングとクレンジング処理関数定義

In [1]:
import re
import unicodedata

from bs4 import BeautifulSoup

# translateで指定する変換法則を指定
translation_table = str.maketrans(dict(zip('()!', '（）！')))

# クレンジング処理
def cleanse(text):
#   NFKC形式でtextを変換、さらに、translateにtranslation_tableを指定することでdict内に従って変換する
    text = unicodedata.normalize('NFKC', text).translate(translation_table)
#   正規表現を使用し\s+（ひとつ以上の空白文字の連続）を空白一つに変換
    text = re.sub(r'\s+', ' ', text)
    return text

# スクレイピング処理
def scrape(html):
    soup = BeautifulSoup(html, 'html.parser')
    # __EOS__ の挿入
    for block in soup.find_all(['br', 'p', 'h1', 'h2', 'h3', 'h4']):
        # strip()で改行を取り除いたブロック内テキストの文字数が0以上 かつ 後ろから一文字目が'。', '！'でない場合
        if len(block.text.strip()) > 0 and block.text.strip()[-1] not in ['。', '！']:
            block.append('<__EOS__>')
            
    # 本文の抽出
    text = '\n'.join([
        # クレンジング処理を行う
        cleanse(block.text.strip()) for block in soup.find_all(['p', 'h1', 'h2', 'h3', 'h4']) if len(block.text.strip()) > 0])
    
    # タイトルの抽出
    # ' - Wikipedia'を削除しcleanse
    title = cleanse(soup.title.text.replace(' - Wikipedia', ''))
    
    return text, title

- ### DB操作関数定義

In [2]:
import json
import sqlite3

conn = None

# データベース接続
def connect():
    # global変数でconnを呼び出し
    global conn
    # データベースの場所を指定
    conn = sqlite3.connect('../data/sqlite3/sqlite3')

# データベース接続終了
def close():
#   終了
    conn.close()

# テーブル作成
def create_table():
#   executeでSQL構文作成、docsがあれば削除
    conn.execute('DROP TABLE IF EXISTS docs')
#   docsテーブルを新規作成
    conn.execute('''CREATE TABLE docs (
            id          INTEGER PRIMARY KEY AUTOINCREMENT,
            content     TEXT,
            meta_info   BLOB,
            sentence    BLOB,
            chunk       BLOB,
            token       BLOB
        )''')

# データをインサートする
def db_load(values):
    
#   valuesに指定したパラメータ順序またはマッピングを?に入れて実行する
    conn.executemany(
        'INSERT INTO docs (content, meta_info) VALUES (?,?)',
        values)
#   確定
    conn.commit()

# 一部のデータを見る
def get(doc_id, fl):
#   .fetchone()でカーソルの次の行を取得
    row_ls = conn.execute(f"SELECT {','.join(fl)} FROM docs WHERE id = {doc_id}").fetchone()
    row_dict = {}
#   flとrow_lsで抜き出したデータをzipする
    for key, value in zip(fl, row_ls):
        row_dict[key] = value
    return row_dict

# id番号を抜き出す
def get_all_ids(limit, offset=0):
#   limitで取得上限、OFFSETで開始位置を指定してデータを抜き出す。そのデータの1番目id番号を抜き出す
    return [record[0] for record in conn.execute('SELECT id FROM docs LIMIT ? OFFSET ?', (limit, offset))]

# アノテーションを設定
def set_annotation(doc_id, name, value):
#   docsのid行をwhere idで指定し、その行にname = valueのアノテーションを追加　
    conn.execute('UPDATE docs SET {0} = ? where id = ?'.format(name), (json.dumps(value), doc_id))
#   確定
    conn.commit()

# アノテーションを確認
def get_annotation(doc_id, name):
#   docsのid行をwhere idで指定しnameから取り出す
    row = conn.execute('SELECT {0} FROM docs WHERE id = ?'.format(name), (doc_id,)).fetchone()
    if row[0] is not None:
        return json.loads(row[0])
    else:
        return []

- ### solrに登録、検索、アノテーション付関数定義　-searchから未解読

In [3]:
import json
import urllib.parse
import urllib.request

# 使用するSolrのURL
solr_url = 'http://localhost:8983/solr'
# build_openerはログインが必要なサイトのときに使用する
opener = urllib.request.build_opener(urllib.request.ProxyHandler())

# Solrにデータを登録する関数、引数dataは登録するdataをdictで指定
def solr_load(collection, data):
    
    # Solrのコアに対してデータを登録するリクエストを作成,collectionにはデータ登録先のコア名を指定している
    url='{0}/{1}/update'.format(solr_url, collection)
    # Requestインスタンスの作成,
    req = urllib.request.Request(
        url,
        # dataをdumps()でutf-8にエンコード
        data=json.dumps(data).encode('utf-8'),
        headers={'content-type': 'application/json'})

    # データの登録を実行
    print(url)
    # resでリクエストの返答を受け取る
    with opener.open(req) as res:
        # データ確認
        print(res.read().decode('utf-8'))

    # Solrのコアに対してコミット指示するリクエストを作成,collectionにはデータ登録先のコア名を指定している
    url = '{0}/{1}/update?softCommit=true'.format(solr_url, collection)
    # urlに対してリクエスト
    req = urllib.request.Request(url)
    # resuでリクエストの返答を受け取る、opnerはプロキシ環境変数に設定している場合も動くようにする為
    with opener.open(req) as res:
        # データを確認
        print(res.read().decode('utf-8'))


def search(keywords, rows=100):
    query = ' AND '.join([
        '(' + ' OR '.join(['content_txt_ja:"{}"'.format(keyword)
                           for keyword in group]) + ')'
        for group in keywords])
    data = {
        'q':     query,
        'wt':    'json',
        'rows':  rows,
        'hl':    'on',
        'hl.fl': 'content_txt_ja',
    }
    # 検索リクエストの作成（＊１）
    req = urllib.request.Request(
        url='{}/doc/select'.format(solr_url),
        data=urllib.parse.urlencode(data).encode('utf-8'),
    )
    # 検索リクエストの実行（＊２）
    with opener.open(req) as res:
        return json.loads(res.read().decode('utf-8'))


def search_annotation(fl_keyword_pairs, rows=100):
    query = ' AND '.join([
        '(' + ' OR '.join(['{0}:"{1}"'.format(fl, keyword)
                           for keyword in group]) + ')'
        for fl, keywords in fl_keyword_pairs
            for group in keywords])
    data = {
        'q':    query,
        'wt':   'json',
        'rows': rows,
    }
    # 検索リクエストの作成（＊１）
    req = urllib.request.Request(
        url='{}/anno/select'.format(solr_url),
        data=urllib.parse.urlencode(data).encode('utf-8'),
    )
    # 検索リクエストの実行（＊２）
    with opener.open(req) as res:
        return json.loads(res.read().decode('utf-8'))

- ### テーブル作成

In [4]:
# データベースの関数を実行する
connect()
create_table()
close()

- ### DBにwikipediaデータを登録する

In [5]:
import glob
# 引数に指定されたパターンにマッチするファイルパス名を取得するモジュール
import json
import urllib.parse

values = []
# 引数に指定されたパターンにマッチするファイルパス名を取得しfor文で回す
for filename in glob.glob('../data/wikipedia/*.html'):
    with open(filename) as fin:
        # filenameの読み込み
        html = fin.read()
        # スクレイプ関数でスクレイピング処理
        text, title = scrape(html)
        # 下記でスクレイピング処理の確認
        # print('scraped:', title)
        url = 'https://ja.wikipedia.org/wiki/{0}'.format(urllib.parse.quote(title))
        values.append((text, json.dumps({'url': url, 'title': title})))

print('scraped finish')
connect()
db_load(values)

print(list(get_all_ids(limit=-1)))
close()

scraped finish
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197]


- ### Solrにコアを作成する

In [6]:
# ! /vagrant/NaturalLanguageProcessing/solr-8.2.0/bin/solr create -c doc

         To turn off: bin/solr config -c doc -p 8983 -action set-user-property -property update.autoCreateFields -value false

ERROR: 
Core 'doc' already exists!
Checked core existence using Core API command:
http://localhost:8983/solr/admin/cores?action=STATUS&core=doc



- ### spliteデータをsolrに格納

In [8]:
import json

# データベースに接続
connect()
data = []
# id番号を抜き出し回す
for doc_id in get_all_ids(limit=-1):
    row = get(doc_id, ['id', 'content', 'meta_info'])
    # Solr へ登録するデータ構造へ変換
    meta_info = json.loads(row['meta_info'])
    data.append({
        'id':               str(row['id']),
        'doc_id_i':         row['id'],
        'content_txt_ja':   row['content'],
        'title_txt_ja':     meta_info['title'],
        'url_s':            meta_info['url'],
    })
# Solr への登録を実行
solr_load('doc', data)
close()

http://localhost:8983/solr/doc/update
{
  "responseHeader":{
    "status":0,
    "QTime":3482}}

{
  "responseHeader":{
    "status":0,
    "QTime":832}}



### solrにアクセスする
* ブラウザで下記にアクセス
```
http://localhost:8983/solr
```
* 「Core Selector」-「doc」を選択
* 「query」を選択
* qの欄に「content_txt_ja:"石油ガス" and content_txt_ja:"天然ガス"」
* Execute Queryを実行